In [1]:
from skbeam.core import utils
import numpy as np

In [12]:
def circular_average1(image, calibrated_center, threshold=0, nx=100,
                     pixel_size=(1, 1),  min_x=None, max_x=None, mask=None):
    """Circular average of the the image data
    The circular average is also known as the radial integration
    Parameters
    ----------
    image : array
        Image to compute the average as a function of radius
    calibrated_center : tuple
        The center of the image in pixel units
        argument order should be (row, col)
    threshold : int, optional
        Ignore counts below `threshold`
        default is zero
    nx : int, optional
        number of bins in x
        defaults is 100 bins
    pixel_size : tuple, optional
        The size of a pixel (in a real unit, like mm).
        argument order should be (pixel_height, pixel_width)
        default is (1, 1)
    min_x : float, optional number of pixels
        Left edge of first bin defaults to minimum value of x
    max_x : float, optional number of pixels
        Right edge of last bin defaults to maximum value of x
    mask : mask for 2D data. Assumes 1 is non masked and 0 masked.
        None defaults to no mask.
    
    Returns
    -------
    bin_centers : array
        The center of each bin in R. shape is (nx, )
    ring_averages : array
        Radial average of the image. shape is (nx, ).

    See Also
    --------
    bad_to_nan_gen : Create a mask with np.nan entries

    Notes
    -----
    """
    radial_val = utils.radial_grid(calibrated_center, image.shape, pixel_size)

    if mask is not None:
        w = np.where(mask == 1)
        radial_val = radial_val[w]
        image = image[w]
    bin_edges, sums, counts = utils.bin_1D(np.ravel(radial_val),
                                           np.ravel(image), nx,
                                           min_x=min_x,
                                           max_x=max_x)
    th_mask = counts > threshold
    ring_averages = sums[th_mask] / counts[th_mask]

    bin_centers = utils.bin_edges_to_centers(bin_edges)[th_mask]

    return bin_centers, ring_averages 


In [13]:
def circular_average2(image, calibrated_center, threshold=0, nx=100,
                     pixel_size=(1, 1),  min_x=None, max_x=None, mask=None):
    """Circular average of the the image data
    The circular average is also known as the radial integration
    Parameters
    ----------
    image : array
        Image to compute the average as a function of radius
    calibrated_center : tuple
        The center of the image in pixel units
        argument order should be (row, col)
    threshold : int, optional
        Ignore counts below `threshold`
        default is zero
    nx : int, optional
        number of bins in x
        defaults is 100 bins
    pixel_size : tuple, optional
        The size of a pixel (in a real unit, like mm).
        argument order should be (pixel_height, pixel_width)
        default is (1, 1)
    min_x : float, optional number of pixels
        Left edge of first bin defaults to minimum value of x
    max_x : float, optional number of pixels
        Right edge of last bin defaults to maximum value of x
    mask : mask for 2D data. Assumes 1 is non masked and 0 masked.
        None defaults to no mask.
    
    Returns
    -------
    bin_centers : array
        The center of each bin in R. shape is (nx, )
    ring_averages : array
        Radial average of the image. shape is (nx, ).

    See Also
    --------
    bad_to_nan_gen : Create a mask with np.nan entries

    Notes
    -----
    """
    radial_val = utils.radial_grid(calibrated_center, image.shape, pixel_size)

    if mask is None:
        mask = np.ones(image.shape,dtype=bool)
    else:
        mask = mask.astype(bool) #ensure it's the right type

    radial_val = radial_val[mask]
    image = image[mask]

    bin_edges, sums, counts = utils.bin_1D(radial_val,
                                           image, nx,
                                           min_x=min_x,
                                           max_x=max_x)
    th_mask = counts > threshold
    ring_averages = sums[th_mask] / counts[th_mask]

    bin_centers = utils.bin_edges_to_centers(bin_edges)[th_mask]

    return bin_centers, ring_averages 


In [14]:
# setup
N = 1000
r0 = N/2, N/2
img = np.random.random((N,N))
mask = np.random.random((N,N)) > .5
Nrepeats = 10


In [17]:
print("timing first")
from time import time as ttime
t0_i = ttime()

for i in range(Nrepeats):
    bcens, ravgs = circular_average1(img, r0, mask=mask,min_x=10, max_x = 1400, nx=1300)
t0_f = ttime()

print("timing second")
t1_i = ttime()
for i in range(Nrepeats):
    bcens, ravgs = circular_average2(img, r0, mask=mask,min_x=10, max_x = 1400, nx=1300)
t1_f = ttime()

dt0 = (t0_f-t0_i)/Nrepeats
dt1 = (t1_f-t1_i)/Nrepeats

print("Time circavg1: {}ms".format(dt0*1e3))
print("Time circavg2: {}ms".format(dt1*1e3))


timing first
timing second
Time circavg1: 166.363263130188ms
Time circavg2: 147.6438283920288ms
